# Data acquisition and pre-processing of satellite precipitation data

## Before starting, follow this four steps to run the notebook on Google Colab

Step 1. Clone GitHub repository with notebooks and data for the course

In [ ]:
!git clone -- https://github.com/paulmunozpauta/ML_course_IUPWARE2025.git

Step 2: Let's move to the cloned folder

In [ ]:
ls

In [ ]:
%cd ML_course_IUPWARE2025

In [ ]:
ls

Step 3: Set-up environment for running course's code

In [ ]:
# Install Poetry
!pip install poetry
# Disable virtual environment creation (needed for Colab)
!poetry config virtualenvs.create false

If session restarts, repeat Steps 2-3.
If not, move to Step 4


Step 4: Install required packages for the course

In [ ]:
!poetry install --no-root

Now let's start wit the hands-on course

# Software for accessing free satellite data repositories

<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta//ML_course_IUPWARE2025/blob/main/notebooks/static/imgs/FileZilla_download.png?raw=1"></div>

## Start FileZilla

# IMERG satellite precipitation

Website: [https://gpm.nasa.gov/data/imerg](https://gpm.nasa.gov/data/imerg)

## Available Subproducts:
- **Early Run**
- **Late Run**
- **Final Run**

We will focus on the subproduct with the finest spatial/temporal resolution, **IMERG-Early Run**.

## Prior registration to download IMERG data

To access and download IMERG data, prior registration is required. Follow these steps:

https://registration.pps.eosdis.nasa.gov/registration/newContact.html

<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta//ML_course_IUPWARE2025/blob/main/notebooks/static/imgs/IMERG_registration.png?raw=1"></div>


Once registered, confirm your account via the email verification link.  

## Connect to the IMERG data repository

Enter the following information to connect to the IMERG FTP server:

- **Protocol:** FTP - File Transfer Protocol  
- **Host:** `jsimpsonftps.pps.eosdis.nasa.gov`  
- **Logon Type:** Normal  
  - Enter your **username** and **password** from your NASA Earthdata account.

Once connected, navigate to the desired directory to access IMERG data.

<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta//ML_course_IUPWARE2025/blob/main/notebooks/static/imgs/IMERG_FTP.png?raw=1"></div>

Upon connecting, you gain remote access to the IMERG data folder.  

From here, you can navigate through the directories to locate and download the desired datasets.

<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta//ML_course_IUPWARE2025/blob/main/notebooks/static/imgs/IMERG_server.png?raw=1"></div>

## Download data from the IMERG-Early Run repository


Right-click on the folder/file you want to download and select **"Download"** to save it to your local directory.  

## Open an IMERG precipitation file

Define IMERG-ER data folder

In [ ]:
folder = os.getcwd()
folder_files= folder+'/notebooks/data/IMERG-ER/Hourly/Global/202301/'
folder_files

Create a list of precipitation files. Data contains satellite precipittaion for January 31, 2023.

In [ ]:
list_of_Files = glob.glob(folder_files + '/*H5')
list_of_Files.sort()

Display the list

In [ ]:
list_of_Files

Define the path of a file for reading.

In [ ]:
item=folder_files+'3B-HHR-E.MS.MRG.3IMERG.20230131-S000000-E002959.0000.V06C.RT-H5'
item

The name coding '3B-HHR-E.MS.MRG.3IMERG.20230131-S000000-E002959.0000.V06C.RT-H5' indicates:
- 3B-HHR-E.MS.MRG.3IMERG: product information
- 20230131: YearMonthDay
- S000000: start of scan (hhmmss)
- E002959: end of scan (hhmmss)

Open a single precipitation file.

In [ ]:
data = h5py.File( item, 'r' )
precip = data['/Grid/precipitationCal'][:]
precip = np.flip( precip[0,:,:].transpose(), axis=0 )

Display the read data matrix.

In [ ]:
precip

Basic statistics

In [ ]:
print('Maximum precipitation = ',precip.max(),'mm')
print('Average precipitation = ',precip.mean(),'mm')
print('Minimum precipitation = ',precip.min(),'mm')

Display the global precipitation information.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(precip, vmin=-1, vmax=80, extent=[-180, 180, -90, 90])
cbar = plt.colorbar(im, orientation='horizontal')
cbar.set_label('millimeters/hour')
for lon in np.arange(-90, 90+1, 90):
    plt.plot((lon, lon), (-90, +90), color="black", linewidth=1)
for lat in np.arange(-60, 60+1, 30):
    plt.plot((-180, +180), (lat, lat), color="black", linewidth=1)
plt.show()

Georeference and clip the global precipitation data to the mountain catchment

In [ ]:
precip

In [ ]:
lat = data['/Grid/lat'][:]
lon = data['/Grid/lon'][:]
catchment_shp_1 = gpd.read_file(folder+'/notebooks/shapefiles/Catchment_SA.shp')
data = xr.DataArray(data=precip, dims=["lat", "lon"], coords=[lat,lon])
data.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
data.rio.write_crs("epsg:4326", inplace=True)
data_catchment_1= data.rio.clip(catchment_shp_1.geometry.apply(mapping),catchment_shp_1.crs,all_touched=True)

Plot the precipitation for the mountain catchment

In [ ]:
fig = plt.subplots(figsize=(8,8))
data_catchment_1.plot(cmap='cividis',vmin=0,vmax=0.01)
plt.title("IMERG-ER precipitation")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

Basic statistics

In [ ]:
print('Maximum precipitation = ', np.round(data_catchment_1.max().values,2),'mm')
print('Average precipitation = ', np.round(data_catchment_1.mean().values,2),'mm')
print('Minimum precipitation = ', np.round(data_catchment_1.min().values,2),'mm')

Plot the precipitation for the coastal catchment

In [ ]:
catchment_shp_2 = gpd.read_file(folder+'/notebooks/shapefiles/Coastal_catchment.geojson')
data_catchment_2 = data.rio.clip(catchment_shp_2.geometry.apply(mapping),catchment_shp_2.crs,all_touched=True)
fig = plt.subplots(figsize=(8,8))
data_catchment_2.plot(cmap='cividis',vmin=0,vmax=5)
plt.title("Precipitación Global IMERG-ER")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()
print('Maximum precipitation = ', np.round(data_catchment_2.max().values,2),'mm')
print('Average precipitation = ', np.round(data_catchment_2.mean().values,2),'mm')
print('Minimum precipitation = ', np.round(data_catchment_2.min().values,2),'mm')

Read and process all IMERG-ER files. in the folder

Display the list

In [ ]:
list_of_Files

Accumulate precipitation

In [ ]:
data_sum = xr.DataArray(data=np.empty((1800, 3600)), dims=["lat", "lon"])
for index, item in enumerate(list_of_Files):
    print(index+1, 'out of', len(list_of_Files))
    try:
        data = h5py.File(item, 'r')
        precip = data['/Grid/precipitationCal'][:]
        precip[precip < 0] = np.nan
        precip = np.flip(precip[0,:,:].transpose(), axis=0)
        theLats = data['Grid/lat'][:]
        theLons = data['Grid/lon'][:]
        x, y = np.meshgrid(theLons, theLats)

        precip = xr.DataArray(precip, dims=('lat', 'lon'), coords={'lat' : theLats, 'lon' : theLons})

        data_sum += precip

        data.close()

    except (IOError, EOFError) as e:
        continue

Plot the accumulated precipitation information for January 31, 2023.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(data_sum, vmin=-1, vmax=1400, extent=[-180, 180, -90, 90])
cbar = plt.colorbar(im, orientation='horizontal')
cbar.set_label('millimeters/hour')

for lon in np.arange(-90, 90+1, 90):
    plt.plot((lon, lon), (-90, +90), color="black", linewidth=1)
for lat in np.arange(-60, 60+1, 30):
    plt.plot((-180, +180), (lat, lat), color="black", linewidth=1)

plt.show()

Clip that information to the mountain catchment

In [ ]:
data_sum.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
data_sum.rio.write_crs("epsg:4326", inplace=True)
data_sum_catchment_1= data_sum.rio.clip(catchment_shp_1.geometry.apply(mapping),catchment_shp_1.crs,all_touched=True)
data_sum_catchment_1

In [ ]:
fig = plt.subplots(figsize=(8,8))
data_sum_catchment_1.plot(cmap='cividis',vmin=0,vmax=16)
plt.title("IMERG-ER precipitation")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()
print('Maximum precipitación = ', np.round(data_sum_catchment_1.max().values,2),'mm')
print('Average precipitación = ', np.round(data_sum_catchment_1.mean().values,2),'mm')
print('Minimum precipitation = ', np.round(data_sum_catchment_1.min().values,2),'mm')

Now for the coastal catchment

In [ ]:
data_sum = xr.DataArray(data_sum, dims=('lat', 'lon'), coords={'lat' : theLats, 'lon' : theLons})
data_sum.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
data_sum.rio.write_crs("epsg:4326", inplace=True)
data_sum_catchment_2= data_sum.rio.clip(catchment_shp_2.geometry.apply(mapping),catchment_shp_2.crs,all_touched=True)
fig = plt.subplots(figsize=(8,8))
data_sum_catchment_2.plot(cmap='cividis',vmin=0,vmax=9)
plt.title("IMERG-ER precipitation")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()
print('Maximum precipitation = ', np.round(data_sum_catchment_2.max().values,2),'mm')
print('Average precipitation = ', np.round(data_sum_catchment_2.mean().values,2),'mm')
print('Minimum precipitation = ', np.round(data_sum_catchment_2.min().values,2),'mm')

Generate a database with precipitation time series (pixels) for the mountain catchment

In [ ]:
dataset_list = []  # List to store DataFrames for concatenation later
for index, item in enumerate(list_of_Files):
    print(index + 1, 'out of', len(list_of_Files))
    try:
        with h5py.File(item, 'r') as data:
            precip = data['/Grid/precipitationCal'][:]
    except (IOError, EOFError):
        continue  # Skip file if there's an error
    precip[precip < 0] = np.nan  # Replace negative values with NaN
    precip = np.flip(precip[0, :, :].transpose(), axis=0)  # Flip and transpose
    # Convert to xarray DataArray
    precip = xr.DataArray(precip, dims=('lat', 'lon'), coords={'lat': theLats, 'lon': theLons})
    precip.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    precip.rio.write_crs("epsg:4326", inplace=True)
    # Clip the data using the catchment shape
    precip = precip.rio.clip(catchment_shp_1.geometry.apply(mapping), catchment_shp_1.crs, all_touched=True)
    precip = precip.values.flatten()
    # Extract date from filename
    year = int(item[-40:-36])
    month = int(item[-36:-34])
    day = int(item[-34:-32])
    hour = int(item[-22:-20])
    minute = int(item[-20:-18])
    date = datetime.datetime(year, month, day, hour, minute, 0)
    # Convert data to DataFrame
    data = pd.DataFrame(precip)
    data = data.dropna()
    data = data.T
    data.index = pd.DatetimeIndex([date])  # Set datetime index
    dataset_list.append(data)  # Append DataFrame to list
# Concatenate all DataFrames at the end
if dataset_list:
    dataset = pd.concat(dataset_list)
# Resample dataset, treating NaNs as zeros
dataset = dataset.resample('h', label='right', closed='right').sum().fillna(0)

Display dataframe

In [ ]:
dataset

Export the dataframe to csv

In [ ]:
dataset.to_csv(folder_files+"IMERG-ER_UTC_Catchment_1.csv", index=True)

Plot the precipitation time series.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dataset.plot(kind='bar', ax=ax)
ax.legend(title='Legend Title')
plt.ylabel('Precipitation_satellite (mm)')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=7)
plt.show()

Plot the accumulated precipitation

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dataset.cumsum().plot( ax=ax)
ax.legend(title='Legend Title')
plt.ylabel('Precipitation_satellite (mm)')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=7)
plt.show()

Do the same for the coastal catchment

In [ ]:
dataset_list = []  # Store data frames for concatenation later
for index, item in enumerate(list_of_Files):
    print(index + 1, 'out of', len(list_of_Files))
    try:
        with h5py.File(item, 'r') as data:
            precip = data['/Grid/precipitationCal'][:]
    except (IOError, EOFError):
        continue  # Skip to next file if there's an error
    precip[precip < 0] = np.nan  # Replace negative values with NaN
    precip = np.flip(precip[0, :, :].transpose(), axis=0)  # Flip and transpose
    # Convert to xarray DataArray
    precip = xr.DataArray(precip, dims=('lat', 'lon'), coords={'lat': theLats, 'lon': theLons})
    precip.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    precip.rio.write_crs("epsg:4326", inplace=True)
    # Clip the data using the catchment shape
    precip = precip.rio.clip(catchment_shp_2.geometry.apply(mapping), catchment_shp_2.crs, all_touched=True)
    precip = precip.values.flatten()
    # Extract date from filename
    year = int(item[-40:-36])
    month = int(item[-36:-34])
    day = int(item[-34:-32])
    hour = int(item[-22:-20])
    minute = int(item[-20:-18])
    date = datetime.datetime(year, month, day, hour, minute, 0)
    # Convert to DataFrame
    data = pd.DataFrame(precip)
    data = data.dropna()
    data = data.T
    data.index = pd.DatetimeIndex([date])  # Set datetime index
    dataset_list.append(data)  # Append to the list
# Concatenate all DataFrames
if dataset_list:
    dataset = pd.concat(dataset_list)
# Resample dataset
dataset = dataset.resample('h', label='right', closed='right').sum()

Display the dataframe

In [ ]:
dataset

Export the dataframe to csv

In [ ]:
dataset.to_csv(folder_files+"IMERG-ER_UTC_Catchment_2.csv", index=True)

Plot the precipitation time series.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dataset.plot(kind='bar', ax=ax)
ax.legend(title='Legend Title')
plt.ylabel('Precipitation_satellite (mm)')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=9)
plt.show()

Plot the accumulated precipitation

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dataset.cumsum().plot( ax=ax)
ax.legend(title='Legend Title')
plt.ylabel('Precipitation_satellite (mm)')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=9)
plt.show()